In [62]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import KFold,StratifiedKFold

In [117]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [85]:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')

In [86]:
ADD_FEATURES = True
USE_STACKING = False
WITH_ORG_FEATURES = True

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [87]:
X.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [88]:
sc = StandardScaler()
sc.fit(X)
X = pd.DataFrame(sc.transform(X), columns=X.columns.values)
X_test = pd.DataFrame(sc.transform(X_test), columns=X_test.columns.values)

In [89]:
X.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,-0.577102,-1.273737,0.451707,-0.833709,0.235571,-0.536430,-0.334926,0.608751,-1.561580,-1.473796,...,0.263374,-1.149158,0.817469,-0.411013,0.168705,-1.578117,1.022131,-0.373968,-1.026398,0.214135
1,0.269959,-0.622138,1.190360,-0.688846,0.790975,1.539900,0.244461,-0.003525,0.858974,0.419300,...,0.966611,0.093605,0.443623,1.908764,-0.817594,1.522342,1.067654,-0.129400,0.825417,0.505685
2,-0.681113,-0.276066,0.516988,0.536516,-0.305477,-0.511033,1.769839,-0.564749,-1.561370,-1.307408,...,-0.072093,0.777997,-0.174131,-0.412316,1.151591,2.297370,-1.617906,-0.695141,-0.381449,0.356681
3,0.125158,-0.129426,-0.667575,0.195355,0.927992,0.410672,0.500633,-0.474201,-1.843910,0.548767,...,0.270216,-0.891456,-0.818468,-0.478548,1.607869,-0.789517,-0.959020,1.501744,0.697118,-0.543502
4,-0.277303,0.035610,0.817683,-0.077829,0.738607,0.955574,0.613372,0.791544,1.794753,0.090006,...,-1.036191,0.688988,-1.405987,1.468536,-1.501101,-0.958473,0.297627,0.645537,0.706318,-0.525375


In [90]:
X_test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0.126869,2.322801,0.847597,1.288432,0.218321,0.341439,0.508134,0.503691,0.554979,1.006232,...,-1.182050,1.459049,-2.271236,-0.220684,-1.365988,1.825273,0.378058,1.965925,-0.132358,-0.516686
1,-0.707067,0.711580,0.223223,-0.788293,-1.158820,0.133993,0.704648,0.610213,-1.409478,-1.290081,...,1.618905,0.461915,-0.667969,1.702503,-0.771026,0.433330,-0.694589,1.045159,1.082247,-1.690887
2,-1.709586,-2.155656,-0.217537,0.123021,-0.502440,1.891142,-0.593060,1.084780,0.371820,0.629086,...,-0.873447,1.175912,0.035485,-0.288519,-1.598800,1.588245,-1.725582,-2.003376,1.336753,-1.901982
3,-0.704764,0.075412,0.494837,-0.108466,-1.375436,1.047877,-0.541479,1.176147,0.927599,-0.088608,...,1.389448,0.541862,-0.182518,0.062506,-0.895134,2.314090,0.303127,0.348670,-0.947851,-0.084716
4,0.337458,0.369113,1.292864,0.466923,-1.216671,-0.447586,1.673832,-1.738131,0.811624,-0.342926,...,0.217430,0.573516,-0.435829,0.011530,0.497716,-0.100682,-1.368546,-1.757722,-0.645885,-0.561225


In [119]:
def add_feature(o, r, f, s, t):
    if s == t:
        return
    if s > t:
        (s, t) = (t, s)
    name = f + "_" + str(s) + "_" + str(t)
    if f == "mean":
        o[name] = r.iloc[:,range(s,t)].mean(axis=1)
    if f == "max":
        o[name] = r.iloc[:,range(s,t)].max(axis=1)
    if f == "min":
        o[name] = r.iloc[:,range(s,t)].min(axis=1)
    if f == "std":
        o[name] = r.iloc[:,range(s,t)].std(axis=1)
    if f == "sum":
        o[name] = r.iloc[:,range(s,t)].sum(axis=1)
    if f == "mean_dev_std":
        o[name] = r.iloc[:,range(s,t)].mean(axis=1) / r.iloc[:,range(s,t)].std(axis=1)
    if f == "abs_sum_mean":
        o[name] = pd.Series(r.iloc[:,range(s,t)].apply(lambda v: np.sum(np.abs(v))).values) / (t - s)
            
def add_features_both(f, s, t):
    add_feature(test_df, X_test, f, s, t)
    add_feature(train_df, X, f, s, t)

def add_features_all(s, t):
    if t >= 200:
        t = 200
    add_features_both("mean", s, t)
    add_features_both("max", s, t)
    add_features_both("min", s, t)
    add_features_both("std", s, t)
    add_features_both("mean_dev_std", s, t)
    add_features_both("sum", s, t)
    add_features_both("abs_sum_mean", s, t)

In [ ]:
if ADD_FEATURES:
    add_features_all(0, 200)
    for i in range(0,191,5):
        add_features_all(i, i + 5)

In [ ]:
test_df.head()

In [133]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,std_0_200,mean_0_200,max_0_200,min_0_200,mean_dev_std_0_200,mean_0_5,max_0_5,min_0_5,std_0_5,mean_dev_std_0_5,mean_5_10,max_5_10,min_5_10,std_5_10,mean_dev_std_5_10,mean_10_15,max_10_15,min_10_15,std_10_15,mean_dev_std_10_15,mean_15_20,max_15_20,min_15_20,std_15_20,mean_dev_std_15_20,mean_20_25,max_20_25,min_20_25,std_20_25,mean_dev_std_20_25,mean_25_30,max_25_30,min_25_30,std_25_30,mean_dev_std_25_30,mean_30_35,max_30_35,min_30_35,std_30_35,mean_dev_std_30_35,mean_35_40,max_35_40,min_35_40,std_35_40,mean_dev_std_35_40,mean_40_45,max_40_45,min_40_45,std_40_45,mean_dev_std_40_45,mean_45_50,max_45_50,min_45_50,std_45_50,mean_dev_std_45_50,mean_50_55,max_50_55,min_50_55,std_50_55,mean_dev_std_50_55,mean_55_60,max_55_60,min_55_60,std_55_60,mean_dev_std_55_60,mean_60_65,max_60_65,min_60_65,std_60_65,mean_dev_std_60_65,mean_65_70,max_65_70,min_65_70,std_65_70,mean_dev_std_65_70,mean_70_75,max_70_75,min_70_75,std_70_75,mean_dev_std_70_75,mean_75_80,max_75_80,min_75_80,std_75_80,mean_dev_std_75_80,mean_80_85,max_80_85,min_80_85,std_80_85,mean_dev_std_80_85,mean_85_90,max_85_90,min_85_90,std_85_90,mean_dev_std_85_90,mean_90_95,max_90_95,min_90_95,std_90_95,mean_dev_std_90_95,mean_95_100,max_95_100,min_95_100,std_95_100,mean_dev_std_95_100,mean_100_105,max_100_105,min_100_105,std_100_105,mean_dev_std_100_105,mean_105_110,max_105_110,min_105_110,std_105_110,mean_dev_std_105_110,mean_110_115,max_110_115,min_110_115,std_110_115,mean_dev_std_110_115,mean_115_120,max_115_120,min_115_120,std_115_120,mean_dev_std_115_120,mean_120_125,max_120_125,min_120_125,std_120_125,mean_dev_std_120_125,mean_125_130,max_125_130,min_125_130,std_125_130,mean_dev_std_125_130,mean_130_135,max_130_135,min_130_135,std_130_135,mean_dev_std_130_135,mean_135_140,max_135_140,min_135_140,std_135_140,mean_dev_std_135_140,mean_140_145,max_140_145,min_140_145,std_140_145,mean_dev_std_140_145,mean_145_150,max_145_150,min_145_150,std_145_150,mean_dev_std_145_150,mean_150_155,max_150_155,min_150_155,std_150_155,mean_dev_std_150_155,mean_155_160,max_155_160,min_155_160,std_155_160,mean_dev_std_155_160,mean_160_165,max_160_165,min_160_165,std_160_165,mean_dev_std_160_165,mean_165_170,max_165_170,min_165_170,std_165_170,mean_dev_std_165_170,mean_170_175,max_170_175,min_170_175,std_170_175,mean_dev_std_170_175,mean_175_180,max_175_180,min_175_180,std_175_180,mean_dev_std_175_180,mean_180_185,max_180_185,min_180_185,std_180_185,mean_dev_std_180_185,mean_185_190,max_185_190,min_185_190,std_185_190,mean_

In [135]:
cov = train_df.cov()

KeyboardInterrupt: 

In [ ]:
cov.iloc[0,:]

In [74]:
X_tophalf, _, y_tophalf, _ = train_test_split(X, y, test_size=0.8, random_state=10)

In [75]:
if USE_STACKING:
    pred_cv = np.zeros(len(X))
    pred = np.zeros(len(X_test))

In [76]:
if USE_STACKING:
    # results from other models
    models = [
        ('lg', SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.3)),
        ('mlp', MLPClassifier(solver='lbfgs', alpha=0.001, hidden_layer_sizes=(5, 2), random_state=1)),
        ('rf', RandomForestClassifier(n_estimators=10, criterion='entropy')),
        ('gnb', GaussianNB()),
        ('qda', QuadraticDiscriminantAnalysis(tol=1e-12)),
    ]

    train_models_out = pd.DataFrame(index=train_df.iloc[:,0])
    test_models_out = pd.DataFrame(index=test_df.iloc[:,0])

    for model in models:
        name = model[0]
        cls = model[1]
        print('working on ' + name)

        # k-fold
        n_splits = 5
        folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        pred_cv = np.zeros(len(X))
        pred = np.zeros(len(X_test))

        for fold_n, (train_index, valid_index) in enumerate(folds.split(X_tophalf,y_tophalf)):
            print("fold: " + str(fold_n))
            X_train, X_valid = X_tophalf.iloc[train_index], X_tophalf.iloc[valid_index]
            y_train, y_valid = y_tophalf.iloc[train_index], y_tophalf.iloc[valid_index]

            cls.fit(X_train, y_train)
            pred_cv += cls.predict_proba(X)[:,1] / n_splits
            pred += cls.predict_proba(X_test)[:,1] / n_splits

        print(pred_cv)

        train_models_out[name] = pred_cv
        test_models_out[name] = pred

In [77]:
if USE_STACKING:
    # add statistics
    num = len(models);
    train_models_out['models_mean'] = train_models_out.iloc[:,[0,num-1]].mean(axis=1)
    train_models_out['models_min'] = train_models_out.iloc[:,[0,num-1]].min(axis=1)
    train_models_out['models_max'] = train_models_out.iloc[:,[0,num-1]].max(axis=1)
    test_models_out['models_mean'] = test_models_out.iloc[:,[0,num-1]].mean(axis=1)
    test_models_out['models_min'] = test_models_out.iloc[:,[0,num-1]].min(axis=1)
    test_models_out['models_max'] = test_models_out.iloc[:,[0,num-1]].max(axis=1)

In [78]:
if USE_STACKING:
    train_models_out.head()

In [79]:
if USE_STACKING:
    test_models_out.head()

In [80]:
if USE_STACKING:
    # add to data
    for col in train_models_out:
        train_df[col] = train_models_out[col].values
        test_df[col] = test_models_out[col].values

In [81]:
if not WITH_ORG_FEATURES:
    for i in range(0, 200):
        train_df = train_df.drop("var_" + str(i), axis=1)
        test_df = test_df.drop("var_" + str(i), axis=1)

In [82]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,std_0_200
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914,0.968946
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518,1.052822
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965,0.895346
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996,1.020416
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104,0.996451


In [83]:
test_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,std_0_200
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197,0.981499
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760,1.013907
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794,1.081784
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108,1.012352
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846,0.981903


In [84]:
train_df.to_csv("input/train_more_features.csv", index=False)
test_df.to_csv("input/test_more_features.csv", index=False)